# Day 12 Part 1 - Navigation for fools

So this feels quite easy compared to the previous few days.  We basically need to step through the directions keeping track of two variables, our current location, and our heading.  
When we get an absolute direction, we just update our location.
When we get a L/R we update our heading
and when we get a Forward direction, we update our location based on our heading.

Having a glance down the data and the examples, it looks highly likely that we'll only have to handle headings on 90 degree chunks, which means we can probably keep our heading as a simple N,E,S,W, or as 0,90,180,270 depending on how I'm feeling.

We'll also want to calculate the manhatten distance between two points, x1,y1 and x2,y2, which is simple enough, but is probably worth turning into a function.  I've used tuples a lot so far for coordinates, but I'm going to use the more advanced namedtuple this time, which is a little like a Scala case class.  It allows us to treat the tuple object a bit like an object, with an x and y attribute, without all the overhead of classes.  This is because they are read-only, but if our changes return copies with updated locations, we can simply assign those around if we want (or if we need it part 2, keep a list of all the places we went too...).  

In [12]:
import ipytest
ipytest.autoconfig()
from collections import namedtuple

Ship = namedtuple('Ship', ['x', 'y', 'heading'], defaults=(90,))
NORTH=0
EAST=90
SOUTH=180
WEST=270

def distance(ship1, ship2=Ship(0,0)):
    return abs(ship1.x-ship2.x)+abs(ship1.y-ship2.y)

assert distance(Ship(0,0), Ship(10,3)) == 13
assert distance(Ship(5,0), Ship(15,3)) == 13
assert distance(Ship(0,0), Ship(10,-3)) == 13
assert distance(Ship(10,3), Ship(0,0)) == 13

In [13]:
def rotate(ship, amount):
    return Ship(ship.x, ship.y, (ship.heading + amount) % 360)
    
assert rotate(Ship(0,0), 90) == Ship(0,0,SOUTH)
assert rotate(Ship(0,0, SOUTH), 90) == Ship(0,0,WEST)
assert rotate(Ship(0,0, WEST), 90) == Ship(0,0,NORTH)
assert rotate(Ship(0,0, NORTH), 90) == Ship(0,0,EAST)
assert rotate(Ship(0,0), -90) == Ship(0,0,NORTH)
assert rotate(Ship(0,0, NORTH), -90) == Ship(0,0,WEST)

In [15]:
   
def process(ship, instruction):
    dist = int(instruction[1:])
    if instruction[0]== "N":
        return Ship(ship.x, ship.y+dist, ship.heading)
    if instruction[0]== "E":
        return Ship(ship.x+dist, ship.y, ship.heading)
    if instruction[0]== "S":
        return Ship(ship.x, ship.y-dist, ship.heading)
    if instruction[0]== "W":
        return Ship(ship.x-dist, ship.y, ship.heading)
    if instruction[0]=="R":
        return rotate(ship, dist)
    if instruction[0]=="L":
        return rotate(ship, -dist)
    if instruction[0]=="F":
        if ship.heading == NORTH:
            return Ship(ship.x, ship.y+dist, ship.heading)
        if ship.heading == EAST:
            return Ship(ship.x+dist, ship.y, ship.heading)
        if ship.heading == SOUTH:
            return Ship(ship.x, ship.y-dist, ship.heading)
        if ship.heading == WEST:
            return Ship(ship.x-dist, ship.y, ship.heading)
        
ship = Ship(0,0)
ship = process(ship, "F10")
assert ship == Ship(10,0,EAST)
ship = process(ship, "N3")
assert ship == Ship(10,3,EAST)
ship = process(ship, "F7")
assert ship == Ship(17,3,EAST)
ship = process(ship, "R90")
assert ship == Ship(17,3,SOUTH)
ship = process(ship, "F11")
assert ship == Ship(17,-8,SOUTH)

assert distance(ship) == 25

Ok, let's try that on production data

In [ ]:
directions = [line.strip() for line in open("day12.txt").readlines()]
ship = Ship(0,0)
locations = [][
for direction in directions:
    ship = 